In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob

import pickle
from pydantic import BaseModel
from openai import OpenAI

from tenacity import (
                        retry,
                        stop_after_attempt,
                        wait_random_exponential
)

from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/LakeBench'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/LakeBench


In [4]:
@retry(wait=wait_random_exponential(min=1,max=60), stop=stop_after_attempt(6))
def execute_prompt(client, system_msg, user_msg):
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            messages=[
                                                        {
                                                            "role": "system", 
                                                             "content": f"{system_msg}"
                                                        },
                                                        {
                                                            "role": "user",
                                                            "content": f"{user_msg}"
                                                        }
                                                    ]
                                            )
    return completion

In [5]:
client = OpenAI()

In [6]:
prompts = np.load('table_JD_with_columns.npy')
prompts.shape

(1, 2)

In [7]:
table_cartesians = pd.read_csv('table_cartesians.csv')
table_cartesians.shape

(1, 2)

In [8]:
def generate_predictions(system_msg, user_msg, client):
    
    result = execute_prompt(client, system_msg, user_msg)
    jd_prediction = result.choices[0].message.content.split('Answer: ')[-1].strip()
    
    return jd_prediction

In [9]:
prompts[0][1]

'Target Table description: The table "SG_CSV0000000000000925.csv" provides detailed demographic and sociocultural data on individuals residing in Singapore, capturing aspects such as age, gender, occupation, educational level, marital status, income, and social networks, along with participants\' perceptions and rankings on various sociocultural and moral segments.\n                  Target Table columns descriptions: ["The \'respondent_serial\' column contains unique numerical identifiers assigned to each respondent participating in the survey, ensuring each entry is distinctly recognizable within the dataset."\n \'The \\\'respondent_id\\\' column contains unique identifiers assigned to each survey participant, formatted as a string that typically begins with "GEN24_" followed by a number or a combination of characters and numbers, serving to distinguish each respondent within the dataset.\'\n \'The "country" column contains the name of the country where the respondent is from, in thi

In [10]:
asnwers = []

for i in tqdm(range(len(prompts))):

    system_msg, user_msg = prompts[i][0], prompts[i][1]
    result = generate_predictions(system_msg, user_msg, client)
    asnwers.append(result)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.00s/it]


In [11]:
asnwers

['Yes']

In [12]:
table_cartesians['JOINABLE'] = asnwers

In [13]:
table_cartesians

,LEFT_TABLE,RIGHT_TABLE,JOINABLE
0,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,Yes


In [14]:
table_cartesians.to_csv('table_cartesians_JD_with_columns_descriptions.csv', index=False)